# **공연별 예매된 좌석, 예매되지 않은 좌석 테이블 구조 만들기**

In [18]:
import pandas as pd 
import sys
import numpy as np
import time

In [2]:
seat_information = pd.read_csv("C:/Users/whileduck/Desktop/Github/Concert-Hall-Price-Model/data/seat_information.csv", encoding = 'euc-kr')
df = pd.read_csv("C:/Users/whileduck/Desktop/Github/Concert-Hall-Price-Model/data/dataframe_reduced_size.csv")

In [3]:
display(seat_information.tail()) # 마지막 셀에 결측치들이 있으니 drop 해주자
seat_information = seat_information.dropna()

,층,블록,열,넘버,전체_좌석,X,Y,Z
2501,2층,BOX6,없음,3.0,2층 BOX6 3,-15001.0,5646.0,66081.0
2502,2층,BOX6,없음,4.0,2층 BOX6 4,-14540.0,6173.0,66081.0
2503,2층,BOX6,없음,5.0,2층 BOX6 5,-15217.0,6765.0,66631.0
2504,2층,BOX6,없음,6.0,2층 BOX6 6,-15678.0,6238.0,66631.0
2505,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [4]:
# merge 하기 위해 전체_좌석 컬럼을 seat 라는 컬럼 명으로 변경시커주자 

seat_information = seat_information.rename(columns = {'전체_좌석':'seat'})

display(seat_information.head())

,층,블록,열,넘버,seat,X,Y,Z
0,1층,A블록,1,1.0,1층 A블록1열 1,14512.0,5423.0,60874.0
1,1층,A블록,1,2.0,1층 A블록1열 2,14060.0,5549.0,60874.0
2,1층,A블록,1,3.0,1층 A블록1열 3,13607.0,5675.0,60874.0
3,1층,A블록,1,4.0,1층 A블록1열 4,13154.0,5802.0,60874.0
4,1층,A블록,1,5.0,1층 A블록1열 5,12696.0,5906.0,60874.0


In [5]:
print('좌석 정보 데이터 프레임 : ', seat_information.shape)
print('메모리 정리 된 데이터 프레임 :', df.shape)

좌석 정보 데이터 프레임 :  (2505, 8)
메모리 정리 된 데이터 프레임 : (1071895, 33)


# **해야 할 일**

각 공연 일, performance code 별 예매 된 좌석에 대해서는 예매 여부를 담은 테이블을 만들어야 함

In [6]:
print('유니크한 공연일 개수 : ',df['전체공연시간'].nunique())

유니크한 공연일 개수 :  738


만약 공연일 하루마다 하나의 공연만 있다고 하더라도 737 * 2505 (전체 좌석) = 184만행의 큰 데이터가 만들어짐 

공연 장르 별 좌석 좌표를 붙일 예정이기 때문에 테이블 구조를 장르 별로 따로 정리해서 만들자 

In [7]:
# ticket_cancel 의 경우는 예매가 되지 않은 것으로 가정하고 drop 해주도록 함 

cond = df['ticket_cancel'] == 0 

df = df[cond]

# 클래스 내에서 처리하기 힘드니까 drop 시키고 병합 후에 변수 다시 만들자

# **빅 콘테스트 의문사항**

In [8]:
unique_combinations = df.groupby(['genre','performance_code', '전체공연시간']).size().reset_index(name='count')

cond = unique_combinations['count'] > 2505

unique_combinations[cond]

,genre,performance_code,전체공연시간,count
238,교향곡,1498,2023-04-22 11:00:00,2630
239,교향곡,1499,2023-05-27 11:00:00,2624
344,교향곡,2822,2023-07-01 17:00:00,2729
352,교향곡,2830,2023-06-24 17:00:00,2661
368,독주,834,2023-02-25 17:00:00,2540
478,오페라,1150,2019-12-04 20:00:00,2730


전체 공연 좌석이 2505 개인데 2505개보다 많은 좌석이 판매된 경우가 있음

살펴보니 같은 좌석에서 다른 사람이 다른 시간에 예매를 한 것으로 나옴

이렇게 되면 해당 공연의 예매율은 100% 를 넘어갈텐데

우선 그룹핑 할때는 이런 일이 없도록 drop_duplicate 를 시행해주자

# **좌석 좌표와 병합 후 없는 좌석은 예매 여부 0으로 하기**

In [9]:
class seat_merging:
    
    def __init__(self,data,seat):
        '''
        예매 된 공연 테이블과 좌석 정보를 담은 테이블을 가져오기
        '''
        self.df = data 
        self.seat = seat
        
        self.seat['예매여부'] = 1 # merging 후 특정 조건에 따라 예매 여부를 0으로 mapping 하여 수정 할 예정 
        
        
        self.genres_df_dict = {}
        self.genres_merged_dict = {}
        self.genres_num_count = {}
        
    def devide_table_bygenre(self):
        
        self.genres = self.df['genre'].unique()
        
        
        for genre in self.genres:
            
            cond = self.df['genre'] == genre
            cond_df = self.df[cond]
            cond_df = cond_df.copy()

            
            num_concert = cond_df['전체공연시간'].nunique()
            
            print(f'{genre} 의 공연 횟수는 :', num_concert)
            
            self.genres_df_dict[genre] = cond_df
            self.genres_num_count[genre] = num_concert
            
    def merging_seat_information(self):
        
        # 병합 후 채워야 하는 컬럼들 
        
        arr = ['play_date','play_st_time','ticket_cancel','performance_code',
                    'pre_open_date','open_date','genre','place','running_time','intermission',
                    '공연연도','공연월','공연일','공연연월','전체공연시간','층','석','세부좌석']
                
        
        for genre in self.genres: # 장르별로 merge  시키기 
            
            data = self.genres_df_dict[genre]
            
            print('--'  * 20)
            print(genre,'진행중 ... ', end = '\t')
            
            unique_performance = sorted(data['전체공연시간'].unique())
            
            
            merging_df_list = []
            
            for uni in unique_performance: # 장르 별, 공연 별로 merge 시키기
                
                cond = data['전체공연시간'] == uni
                
                cond_df = data[cond]
                
                # 좌석의 중복값을 제거해야함 
                # 전체 거래시간 별로 오름차순 정렬 후 마지막에 중복값중 마지막 중복값 제외 drop 하기
                # 마지막 거래가 실제 거래라고 생각하기로 함
                cond_df = cond_df.sort_values(by = '전체거래시간')
                cond_df = cond_df.drop_duplicates(subset = 'seat', keep = 'last')
                
                merging_df = pd.merge(self.seat[['seat','X','Y','Z','예매여부']], cond_df, on = 'seat', how = 'left')
                
                cond_2 = merging_df['전체공연시간'].isna()
                
                merging_df.loc[cond_2, '예매여부'] = 0 # 예매가 안됐던 경우에 대해서는 예매 여부를 0으로 변경하기 
                
                for fill_col in arr: 
                    if merging_df[fill_col].notna().any():
                        original_value = merging_df.loc[merging_df[fill_col].notna(), fill_col].iloc[0]
                        merging_df[fill_col] = merging_df[fill_col].fillna(original_value)
                    
                merging_df_list.append(merging_df)
                        
                    
            genre_merged = pd.concat(merging_df_list)
            
            self.genres_merged_dict[genre] = genre_merged
            
            print(genre,'완료 !')


        return self.genres_merged_dict

In [10]:
merging_method = seat_merging(df,seat_information)

In [11]:
merging_method.devide_table_bygenre()

독주 의 공연 횟수는 : 42
교향곡 의 공연 횟수는 : 362
클래식 의 공연 횟수는 : 162
오페라 의 공연 횟수는 : 14
합창 의 공연 횟수는 : 70
성악 의 공연 횟수는 : 31
실내악 의 공연 횟수는 : 26
콘서트 의 공연 횟수는 : 18
복합장르 의 공연 횟수는 : 6
기타 의 공연 횟수는 : 3
크로스오버 의 공연 횟수는 : 2
재즈 의 공연 횟수는 : 1
가족극 의 공연 횟수는 : 1


In [12]:
merged_dict = merging_method.merging_seat_information()

----------------------------------------
독주 진행중 ... 	독주 완료 !
----------------------------------------
교향곡 진행중 ... 	교향곡 완료 !
----------------------------------------
클래식 진행중 ... 	클래식 완료 !
----------------------------------------
오페라 진행중 ... 	오페라 완료 !
----------------------------------------
합창 진행중 ... 	합창 완료 !
----------------------------------------
성악 진행중 ... 	성악 완료 !
----------------------------------------
실내악 진행중 ... 	실내악 완료 !
----------------------------------------
콘서트 진행중 ... 	콘서트 완료 !
----------------------------------------
복합장르 진행중 ... 	복합장르 완료 !
----------------------------------------
기타 진행중 ... 	기타 완료 !
----------------------------------------
크로스오버 진행중 ... 	크로스오버 완료 !
----------------------------------------
재즈 진행중 ... 	재즈 완료 !
----------------------------------------
가족극 진행중 ... 	가족극 완료 !


In [13]:
# 결과물 예시 

example = merged_dict['클래식']

print('예매여부가 1인 경우')
display(example[example['예매여부'] == 1].head(3))


print('예매여부가 0인 경우')
display(example[example['예매여부'] == 0].head(3))

예매여부가 1인 경우


,seat,X,Y,Z,예매여부,age,gender,membership_type_1,membership_type_2,membership_type_3,...,member_yn,공연연도,공연월,공연일,공연연월,전체공연시간,전체거래시간,층,석,세부좌석
0,1층 A블록1열 1,14512.0,5423.0,60874.0,1,NaN,NaN,NaN,NaN,NaN,...,N,2018.0,11.0,2018-11-25,2018-11-01,2018-11-25 17:00:00,2018-11-16 15:06:00,1층,A블록1열,1.0
1,1층 A블록1열 2,14060.0,5549.0,60874.0,1,NaN,NaN,NaN,NaN,NaN,...,N,2018.0,11.0,2018-11-25,2018-11-01,2018-11-25 17:00:00,2018-11-16 15:06:00,1층,A블록1열,2.0
2,1층 A블록1열 3,13607.0,5675.0,60874.0,1,NaN,NaN,NaN,NaN,NaN,...,N,2018.0,11.0,2018-11-25,2018-11-01,2018-11-25 17:00:00,2018-11-16 15:06:00,1층,A블록1열,3.0


예매여부가 0인 경우


,seat,X,Y,Z,예매여부,age,gender,membership_type_1,membership_type_2,membership_type_3,...,member_yn,공연연도,공연월,공연일,공연연월,전체공연시간,전체거래시간,층,석,세부좌석
636,1층 D블록12열 6,-8097.0,17391.0,62891.0,0,NaN,NaN,NaN,NaN,NaN,...,NaN,2018.0,11.0,2018-11-25,2018-11-01,2018-11-25 17:00:00,NaN,1층,A블록1열,1.0
661,1층 A블록13열 7,15457.0,16967.0,63098.0,0,NaN,NaN,NaN,NaN,NaN,...,NaN,2018.0,11.0,2018-11-25,2018-11-01,2018-11-25 17:00:00,NaN,1층,A블록1열,1.0
662,1층 A블록13열 8,14971.0,17085.0,63098.0,0,NaN,NaN,NaN,NaN,NaN,...,NaN,2018.0,11.0,2018-11-25,2018-11-01,2018-11-25 17:00:00,NaN,1층,A블록1열,1.0


In [14]:
genres = merging_method.genres


uniq_list = []
shapes = []
num_counts = []
for g in genres: # 잘 체크 되었는지 확인 
    
    genre_df = merged_dict[g]

    grouping_df = genre_df.groupby('전체공연시간').size().reset_index(name = 'count')
    
    unqiue_value = grouping_df['count'].unique()    
    
    uniq_list.append(unqiue_value[0])
    shapes.append(genre_df.shape)
    
result = pd.DataFrame({
    '장르':genres,
    'shape' : shapes,
    '장르 별 공연 횟수': merging_method.genres_num_count.values(),
    '각 공연 별 유니크한 좌석 수' : uniq_list
})

display(result)

,장르,shape,장르 별 공연 횟수,각 공연 별 유니크한 좌석 수
0,독주,"(105210, 37)",42,2505
1,교향곡,"(906810, 37)",362,2505
2,클래식,"(405810, 37)",162,2505
3,오페라,"(35070, 37)",14,2505
4,합창,"(175350, 37)",70,2505
5,성악,"(77655, 37)",31,2505
6,실내악,"(65130, 37)",26,2505
7,콘서트,"(45090, 37)",18,2505
8,복합장르,"(15030, 37)",6,2505
9,기타,"(7515, 37)",3,2505


# **잘 완료 되었고 깃허브에서 연동 할 수 있도록 각 데이터 프레임의 용량을 낮춘 후 저장하자**

In [21]:
class DataExploratioin:
    '''
    데이터 탐색 시 사용 가능한 Class 

    기존 존재하는 프레임워크들을 이용하여 자주 이용하는 프레임워크들을 활용하여 나만의 분석 툴을 만들려고 함 

    데이터 요약, 결측값 처리 등의 내용이 담겨있는 class 
    '''

    def __init__(self, data):
        self.data = data

    def summarize(self):
        '''
        데이터를 초창기에 요약해주는 method
        '''

        cols = self.data.columns

        size = round(sys.getsizeof(self.data) / 1024 ** 2, 2)

        print(f'data size : {size}MB')

        self.result = pd.DataFrame()

        self.result['Dtype'] = self.data.dtypes.values
        self.result['Count'] = self.data.count().values
        self.result['Nunique'] = self.data.nunique().values
        self.result['Missing value'] = self.data.isna().sum().values
        self.result['Missing %'] = [str(round(
            missing / len(self.data), 2) * 100) + '%' for missing in self.result['Missing value']]
        self.result['Most Freq Value'] = self.data.mode().iloc[0].values

        freq_prop = []

        for i, col in enumerate(cols):

            raw_data = self.data.loc[~self.data[col].isna(), col]
            freq_value = self.result['Most Freq Value'].iloc[i]

            prop = np.mean(
                np.array(raw_data == freq_value)
            )

            prop_str = str(round(np.mean(prop) * 100, 1)) + '%'

            if prop_str == 'nan%':
                freq_prop.append(self.result['Missing %'].iloc[i])
            else:
                freq_prop.append(prop_str)

        self.result['Most Freq Value %'] = freq_prop

        self.result['Min'] = self.data.describe(include='all').T['min'].values
        self.result['Max'] = self.data.describe(include='all').T['max'].values
        self.result['Mean'] = self.data.describe(
            include='all').T['mean'].values
        self.result['Median'] = self.data.describe(
            include='all').T['50%'].values
        
        memory = (self.data.memory_usage(deep = True) // 1024 **2).values[1:] # index 의 usage 는 제외하고 보자 

        
        self.result['MB'] = [str(m) + ' mb' for m in memory]
        self.result = self.result.set_index(cols)

        self.result = self.result.fillna('-')

        display(self.result)
    
    
    def progress_bar(self,iterable, total_blocks = 10):
        
        total_items = len(iterable)
        block_size = total_items // total_blocks
        
        for i, item in enumerate(iterable, start=1):
            if i % block_size == 0 or i == total_items:
                progress = (i / total_items) * 100
                blocks = int(progress / (100 / total_blocks))
                empty_blocks = total_blocks - blocks
                progress_bar = '■' * blocks + '▢' * empty_blocks
                print(f"\rProgress: [{progress_bar}] {progress:.2f}%", end='', flush=True)
            yield item
            time.sleep(0.0000001)
    
    def reduce_size(self,iwantdisplay = False):
                
        original_size = round(sys.getsizeof(self.data) / 1024 ** 2,2)
        
        df = self.data.copy()
        
        for col in self.progress_bar(df.columns):
            
            dtp = df[col].dtype
            
            if dtp == 'object':
                df[col] = df[col].astype('category')
            else: # numeric type이면 
                
                if min(df[col]) >= 0 : # 부호가 없다면 unit 으로 변경해줘도 된다.
                    max_value = max(df[col])
                    
                    bits = [8,16,32,64]
                    
                    for bit in bits: # 최소한의 비트로 표현 될 수 있게 dtype 변경 
                        if max_value < 2 ** bit:
                            # 결측치가 있는 경우 astype 으로 변경하지 못하니 결측치를 채워준 후 변경하고 다시 결측치를 채우자 
                            df[col] = df[col].fillna(2 ** bit - 1)
                            df[col] = df[col].astype(f'uint{bit}')
                            df[col] = df[col].replace(2 ** bit - 1, np.NaN)
                            break
                        
                else: # 부호가 있다면 int type 으로 바꿔주자 
                    
                    max_value = max(abs(min(df[col])), max(df[col]))
                    
                    bits = [8,16,32,64]
                    
                    for bit in bits:
                        if max_value < 2 ** bit:
                            df[col] = df[col].fillna(2 ** bit - 1)
                            df[col] = df[col].astype(f'int{bit}')
                            df[col] = df[col].replace(2 ** bit - 1, np.NaN)
                            break
                        
        print('\n')
                        
        after_size = round(sys.getsizeof(df) / 1024 ** 2,2)
        
        # 바꾼 후 결과 보여주기 
        
        if iwantdisplay:    
            after = DataExploratioin(df)
            after.summarize()
            
        print(f'\n {original_size}MB -> {after_size}MB')
            
        return df

In [22]:
file_path = 'C:/Users/whileduck/Desktop/Github/Concert-Hall-Price-Model/data'

for genre, data in merged_dict.items():
    
    ep = DataExploratioin(data)
    
    reduce_data = ep.reduce_size()
    
    reduce_data.to_csv(f'C:/Users/whileduck/Desktop/Github/Concert-Hall-Price-Model/data/{genre}_빈좌석포함데이터.csv', index = False)

Progress: [■■■■■■■■■■] 100.00%


 156.1MB -> 10.15MB
Progress: [■■■■■■■■■■] 100.00%


 1348.51MB -> 83.34MB
Progress: [■■■■■■■■■■] 100.00%


 603.29MB -> 37.22MB
Progress: [■■■■■■■■■■] 100.00%


 51.97MB -> 3.32MB
Progress: [■■■■■■■■■■] 100.00%


 263.18MB -> 14.73MB
Progress: [■■■■■■■■■■] 100.00%


 114.78MB -> 6.79MB
Progress: [■■■■■■■■■■] 100.00%


 96.17MB -> 5.96MB
Progress: [■■■■■■■■■■] 100.00%


 66.61MB -> 4.07MB
Progress: [■■■■■■■■■■] 100.00%


 22.4MB -> 1.68MB
Progress: [■■■■■■■■■■] 100.00%


 11.28MB -> 0.96MB
Progress: [■■■■■■■■■■] 100.00%


 7.15MB -> 0.75MB
Progress: [■■■■■■■■■■] 100.00%


 3.73MB -> 0.62MB
Progress: [■■■■■■■■■■] 100.00%


 3.44MB -> 0.54MB
